In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from tqdm import tqdm

# Ticker symbols for the 12 stocks
tickers = [
    "601398.SS", "601988.SS", "601939.SS", "601288.SS", "600036.SS",
    "601328.SS", "601318.SS", "601628.SS", "601601.SS", "600016.SS",
    "601688.SS", "000776.SZ"
]

# Download historical OHLCV from 2014-01-01 to 2023-12-31
def download_data(ticker):
    data = yf.download(ticker, start="2014-01-01", end="2023-12-31")
    data = data.dropna()
    return data

stock_data = {ticker: download_data(ticker) for ticker in tqdm(tickers)}

  0%|          | 0/12 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 12/12 [00:04<00:00,  2.42it/s]


In [2]:
def compute_features(df):
    df = df.copy()

    # Ensure Close, High, Low are Series (1D)
    close = df["Close"].squeeze()
    high = df["High"].squeeze()
    low = df["Low"].squeeze()
    volume = df["Volume"].squeeze()

    print(f"Close dtype: {type(close)}, shape: {close.shape}")

    # Daily return
    df["Return"] = close.pct_change()

    # 30-day rolling volatility (target)
    df["Volatility"] = df["Return"].rolling(window=30).std()

    # RSI (14 days)
    df["RSI"] = RSIIndicator(close=close, window=14).rsi()

    # Momentum (5 days)
    df["MOM"] = close - close.shift(5)

    # OBV
    df["OBV"] = (np.sign(close.diff()) * volume).fillna(0).cumsum()

    # MACD
    macd = MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD_LINE"] = macd.macd()
    df["MACD_SIGNAL"] = macd.macd_signal()
    df["MACD_HIST"] = macd.macd_diff()

    # Stochastic Oscillator
    stoch = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
    df["STO_K"] = stoch.stoch()           # formerly %K
    df["STO_D"] = stoch.stoch_signal()    # formerly %D

    # Lagged volatilities (t-1 to t-6)
    for i in range(1, 7):
        df[f"Vol_t_{i}"] = df["Volatility"].shift(i)

    # Volatility t+1 (our target)
    df["Vol_target"] = df["Volatility"].shift(-1)

    # Drop rows with NaNs
    df = df.dropna()

    return df

In [3]:
import os
import pickle

feature_data = {}
for ticker in tqdm(tickers):
    feature_data[ticker] = compute_features(stock_data[ticker])

feature_data_path = "china_feature_data.pkl"

if os.path.exists(feature_data_path):
    print("📦 Loading saved feature data from china_feature_data.pkl...")
    with open(feature_data_path, "rb") as f:
        feature_data = pickle.load(f)
else:
    print("⚙️ Computing feature data...")
    feature_data = {ticker: compute_features(stock_data[ticker]) for ticker in tqdm(tickers)}
    with open(feature_data_path, "wb") as f:
        pickle.dump(feature_data, f)
    print("💾 Saved feature data to china_feature_data.pkl")

100%|██████████| 12/12 [00:00<00:00, 80.49it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
⚙️ Computing feature data...


  0%|          | 0/12 [00:00<?, ?it/s]

Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)


100%|██████████| 12/12 [00:00<00:00, 82.62it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2432,)
💾 Saved feature data to china_feature_data.pkl


In [4]:
from arch import arch_model
import warnings

def add_garch_predictions(df, ticker=None, verbose=True):
    df = df.copy()
    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # recommended by arch package

    if verbose:
        print(f"\n🔍 GARCH modeling for {ticker} — total points: {len(returns)}")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor  # rescale

        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = arch_model(train_window, vol='Garch', p=1, q=1, dist='normal', rescale=False)
                model_fit = model.fit(disp="off")
                forecast = model_fit.forecast(horizon=1)
                pred_vol_scaled = np.sqrt(forecast.variance.values[-1][0])
                pred_vol = pred_vol_scaled / scale_factor  # unscale
        except Exception as e:
            if verbose:
                print(f"⚠️ Failed at i={i} — {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → Index {i} | Pred Vol (unscaled): {pred_vol:.5f}")

    full_preds = [np.nan] * window_size + preds
    df["GARCH_pred"] = full_preds

    before = len(df)
    df = df.dropna()
    after = len(df)

    if verbose:
        print(f"✅ Done {ticker} | Rows dropped: {before - after} | Final: {after} rows")

    return df

In [5]:
# === Try loading precomputed garch_data from disk ===
garch_data_path = "china_garch_data.pkl"

if os.path.exists(garch_data_path):
    print("📦 Loading saved GARCH data from china_garch_data.pkl...")
    with open(garch_data_path, "rb") as f:
        garch_data = pickle.load(f)
    print("✅ Loaded GARCH data successfully!")
else:
    print("⚙️ Computing GARCH data from scratch...")
    garch_data = {}
    for ticker in tickers:
        print(f"\n====================== {ticker} ======================")
        garch_data[ticker] = add_garch_predictions(feature_data[ticker], ticker=ticker)

    # Save to disk
    with open(garch_data_path, "wb") as f:
        pickle.dump(garch_data, f)
    print("💾 Saved GARCH data to china_garch_data.pkl")

⚙️ Computing GARCH data from scratch...

====================== 601398.SS ======================

🔍 GARCH modeling for 601398.SS — total points: 2395
  → Index 500 | Pred Vol (unscaled): 0.01529
  → Index 750 | Pred Vol (unscaled): 0.00571
  → Index 1000 | Pred Vol (unscaled): 0.01758
  → Index 1250 | Pred Vol (unscaled): 0.01197
  → Index 1500 | Pred Vol (unscaled): 0.00824
  → Index 1750 | Pred Vol (unscaled): 0.01567
  → Index 2000 | Pred Vol (unscaled): 0.00726
  → Index 2250 | Pred Vol (unscaled): 0.01931
✅ Done 601398.SS | Rows dropped: 500 | Final: 1895 rows

====================== 601988.SS ======================

🔍 GARCH modeling for 601988.SS — total points: 2395
  → Index 500 | Pred Vol (unscaled): 0.01635
  → Index 750 | Pred Vol (unscaled): 0.00773
  → Index 1000 | Pred Vol (unscaled): 0.01273
  → Index 1250 | Pred Vol (unscaled): 0.00911
  → Index 1500 | Pred Vol (unscaled): 0.00980
  → Index 1750 | Pred Vol (unscaled): 0.00710
  → Index 2000 | Pred Vol (unscaled): 0.0050

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def evaluate(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_ml_models_baseline(df, ticker="TICKER"):
    print(f"\n📈 Training ML models for {ticker}...")

    # Feature and target selection
    features = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D',
        'Vol_t_1', 'Vol_t_2', 'Vol_t_3', 'Vol_t_4', 'Vol_t_5', 'Vol_t_6'
    ]

    X = df[features].copy()
    # Sanitize column names just in case LightGBM is sensitive
    X.columns = [str(col).replace("-", "_").replace("%", "PCT").replace(".", "_DOT_") for col in X.columns]

    y = df["Vol_target"]

    # Static train-test split (same as paper: 2014–2020 train, 2021–2023 test)
    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        #"LightGBM": LGBMRegressor(),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics = evaluate(y_test, y_pred)
        results[name] = metrics
        print(f"✅ {name} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return results

/shared/home/ads066/.local/lib/python3.11/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mse),
        "MSE": mse,
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_all_stocks_ml_baseline(garch_data_dict, results_path="china_ml_baseline_results.csv"):
    # Check if results already exist
    if os.path.exists(results_path):
        print(f"📦 Loading existing results from {results_path}...")
        return pd.read_csv(results_path)

    final_results = []

    for ticker, df in garch_data_dict.items():
        print(f"\n================= {ticker} =================")
        results = train_ml_models_baseline(df, ticker=ticker)

        for model_name, metrics in results.items():
            final_results.append({
                "Stock": ticker,
                "Model": model_name,
                "R2": round(metrics["R2"], 4),
                "RMSE": round(metrics["RMSE"], 4),
                "MSE": round(metrics["MSE"], 6),
                "MAE": round(metrics["MAE"], 4),
            })

    # Save results
    results_df = pd.DataFrame(final_results)
    results_df.to_csv(results_path, index=False)
    print(f"💾 Saved results to {results_path}")

    return results_df

# Run training or load existing results
ml_all_results = train_all_stocks_ml_baseline(garch_data)
ml_all_results_sorted = ml_all_results.sort_values(by="R2", ascending=False)
display(ml_all_results_sorted)


================= 601398.SS =================

📈 Training ML models for 601398.SS...
✅ KNN — R²: -2.4385, RMSE: 0.0064, MAE: 0.0055
✅ AdaBoost — R²: 0.7848, RMSE: 0.0016, MAE: 0.0013
✅ CatBoost — R²: 0.9492, RMSE: 0.0008, MAE: 0.0005
✅ XGBoost — R²: 0.9284, RMSE: 0.0009, MAE: 0.0006
✅ RandomForest — R²: 0.9438, RMSE: 0.0008, MAE: 0.0005

================= 601988.SS =================

📈 Training ML models for 601988.SS...
✅ KNN — R²: 0.0187, RMSE: 0.0055, MAE: 0.0042
✅ AdaBoost — R²: 0.8451, RMSE: 0.0022, MAE: 0.0013
✅ CatBoost — R²: 0.8192, RMSE: 0.0023, MAE: 0.0011
✅ XGBoost — R²: 0.8392, RMSE: 0.0022, MAE: 0.0012
✅ RandomForest — R²: 0.8686, RMSE: 0.0020, MAE: 0.0010

================= 601939.SS =================

📈 Training ML models for 601939.SS...
✅ KNN — R²: -2.6311, RMSE: 0.0068, MAE: 0.0059
✅ AdaBoost — R²: 0.7301, RMSE: 0.0019, MAE: 0.0014
✅ CatBoost — R²: 0.8568, RMSE: 0.0014, MAE: 0.0010
✅ XGBoost — R²: 0.8888, RMSE: 0.0012, MAE: 0.0008
✅ RandomForest — R²: 0.9101, RMSE: 0

,Stock,Model,R2,RMSE,MSE,MAE
18,601288.SS,XGBoost,0.9622,0.0008,0.000001,0.0005
2,601398.SS,CatBoost,0.9492,0.0008,0.000001,0.0005
4,601398.SS,RandomForest,0.9438,0.0008,0.000001,0.0005
19,601288.SS,RandomForest,0.9438,0.0010,0.000001,0.0007
3,601398.SS,XGBoost,0.9284,0.0009,0.000001,0.0006
44,601601.SS,RandomForest,0.9124,0.0017,0.000003,0.0011
17,601288.SS,CatBoost,0.9122,0.0013,0.000002,0.0007
14,601939.SS,RandomForest,0.9101,0.0011,0.000001,0.0007
43,601601.SS,XGBoost,0.9091,0.0017,0.000003,0.0012
29,601328.SS,RandomForest,0.9082,0.0012,0.000001,0.0008


In [8]:
from arch import arch_model
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_series(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def forecast_volatility_arch(df, model_type="GARCH", ticker="TICKER", verbose=True):
    df = df.copy()

    # Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if "Vol_target" not in df.columns or "Return" not in df.columns:
        raise KeyError(f"Missing 'Vol_target' or 'Return' in {ticker}")

    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # fix for scale warning

    if verbose:
        print(f"\n🔮 Running {model_type} for {ticker}...")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor

        try:
            if model_type == "GARCH":
                model = arch_model(train_window, vol='GARCH', p=1, q=1, dist='normal', rescale=False)
            elif model_type == "GJR":
                model = arch_model(train_window, vol='GARCH', p=1, o=1, q=1, dist='normal', rescale=False)
            elif model_type == "EGARCH":
                model = arch_model(train_window, vol='EGARCH', p=1, q=1, dist='normal', rescale=False)
            else:
                raise ValueError("Invalid model_type")

            model_fit = model.fit(disp="off")
            forecast = model_fit.forecast(horizon=1)
            pred_vol = np.sqrt(forecast.variance.values[-1][0]) / scale_factor

        except Exception as e:
            if verbose:
                print(f"⚠️ {model_type} failed at index {i}: {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → {model_type} | index {i} | vol: {pred_vol:.5f}")

    df[f"{model_type}_pred"] = [np.nan] * window_size + preds
    df = df.dropna(subset=["Vol_target", f"{model_type}_pred"])

    metrics = evaluate_series(df["Vol_target"], df[f"{model_type}_pred"])
    if verbose:
        print(f"✅ {model_type} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return df, metrics

In [9]:
import os
import pandas as pd

def evaluate_all_series_models(garch_data_dict, results_path="china_ts_model_results.csv"):
    # If results already exist, load them
    if os.path.exists(results_path):
        print(f"📦 Loading saved time series results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for model_type in ["GARCH", "GJR", "EGARCH"]:
            print(f"\n================= {ticker} - {model_type} =================")
            try:
                _, metrics = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=True)
                results.append({
                    "Stock": ticker,
                    "Model": model_type,
                    "R2": round(metrics["R2"], 4),
                    "RMSE": round(metrics["RMSE"], 4),
                    "MSE": round(metrics["MSE"], 6),
                    "MAE": round(metrics["MAE"], 4),
                })
            except Exception as e:
                print(f"⚠️ Skipping {ticker} - {model_type}: {e}")

    df_results = pd.DataFrame(results)
    df_results.to_csv(results_path, index=False)
    print(f"💾 Saved time series model results to {results_path}")

    return df_results

ts_model_results = evaluate_all_series_models(garch_data)
ts_model_results_sorted = ts_model_results.sort_values(by="R2", ascending=False)
display(ts_model_results_sorted)


================= 601398.SS - GARCH =================

🔮 Running GARCH for 601398.SS...
  → GARCH | index 500 | vol: 0.01758
  → GARCH | index 750 | vol: 0.01197
  → GARCH | index 1000 | vol: 0.00824
  → GARCH | index 1250 | vol: 0.01567
  → GARCH | index 1500 | vol: 0.00726
  → GARCH | index 1750 | vol: 0.01931
✅ GARCH for 601398.SS — R²: 0.4657, RMSE: 0.0031, MAE: 0.0022

================= 601398.SS - GJR =================

🔮 Running GJR for 601398.SS...
  → GJR | index 500 | vol: 0.01344
  → GJR | index 750 | vol: 0.01218
  → GJR | index 1000 | vol: 0.00829
  → GJR | index 1250 | vol: 0.01658
  → GJR | index 1500 | vol: 0.00658
  → GJR | index 1750 | vol: 0.01832
✅ GJR for 601398.SS — R²: 0.4465, RMSE: 0.0031, MAE: 0.0022

================= 601398.SS - EGARCH =================

🔮 Running EGARCH for 601398.SS...
  → EGARCH | index 500 | vol: 0.01598
  → EGARCH | index 750 | vol: 0.01242
  → EGARCH | index 1000 | vol: 0.00842
  → EGARCH | index 1250 | vol: 0.01420
  → EGARCH | index 

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → EGARCH | index 1750 | vol: 0.01535
✅ EGARCH for 601398.SS — R²: 0.4655, RMSE: 0.0031, MAE: 0.0022

================= 601988.SS - GARCH =================

🔮 Running GARCH for 601988.SS...
  → GARCH | index 500 | vol: 0.01273
  → GARCH | index 750 | vol: 0.00911
  → GARCH | index 1000 | vol: 0.00980
  → GARCH | index 1250 | vol: 0.00710
  → GARCH | index 1500 | vol: 0.00505
  → GARCH | index 1750 | vol: 0.02613
✅ GARCH for 601988.SS — R²: 0.6200, RMSE: 0.0027, MAE: 0.0019

================= 601988.SS - GJR =================

🔮 Running GJR for 601988.SS...
  → GJR | index 500 | vol: 0.00894
  → GJR | index 750 | vol: 0.00953
  → GJR | index 1000 | vol: 0.01000
  → GJR | index 1250 | vol: 0.00706
  → GJR | index 1500 | vol: 0.00481
  → GJR | index 1750 | vol: 0.02733
✅ GJR for 601988.SS — R²: 0.5024, RMSE: 0.0031, MAE: 0.0020

================= 601988.SS - EGARCH =================

🔮 Running EGARCH for 601988.SS...
  → EGARCH | index 500 | vol: 0.01238
  → EGARCH | index 750 | vol: 0.0

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → GJR | index 750 | vol: 0.01249
  → GJR | index 1000 | vol: 0.00876
  → GJR | index 1250 | vol: 0.00964
  → GJR | index 1500 | vol: 0.00551
  → GJR | index 1750 | vol: 0.01936
✅ GJR for 601288.SS — R²: 0.6249, RMSE: 0.0026, MAE: 0.0018

================= 601288.SS - EGARCH =================

🔮 Running EGARCH for 601288.SS...
  → EGARCH | index 500 | vol: 0.01891


...

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

✅ EGARCH for 601688.SS — R²: -4953661286185424207936298941239263232.0000, RMSE: 14742012936515040.0000, MAE: 394702324584652.2500

================= 000776.SZ - GARCH =================

🔮 Running GARCH for 000776.SZ...
  → GARCH | index 500 | vol: 0.01443
  → GARCH | index 750 | vol: 0.02097
  → GARCH | index 1000 | vol: 0.01442
  → GARCH | index 1250 | vol: 0.01614
  → GARCH | index 1500 | vol: 0.02475
  → GARCH | index 1750 | vol: 0.01767
✅ GARCH for 000776.SZ — R²: 0.6584, RMSE: 0.0049, MAE: 0.0035

================= 000776.SZ - GJR =================

🔮 Running GJR for 000776.SZ...
  → GJR | index 500 | vol: 0.01432
  → GJR | index 750 | vol: 0.02181
  → GJR | index 1000 | vol: 0.01404
  → GJR | index 1250 | vol: 0.01604
  → GJR | index 1500 | vol: 0.02458
  → GJR | index 1750 | vol: 0.01771
✅ GJR for 000776.SZ — R²: 0.6350, RMSE: 0.0050, MAE: 0.0036

================= 000776.SZ - EGARCH =================

🔮 Running EGARCH for 000776.SZ...
  → EGARCH | index 500 | vol: 0.01522
  → E

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/

✅ EGARCH for 000776.SZ — R²: -31.1433, RMSE: 0.0472, MAE: 0.0054
💾 Saved time series model results to china_ts_model_results.csv


,Stock,Model,R2,RMSE,MSE,MAE
33,000776.SZ,GARCH,6.584000e-01,4.900000e-03,2.400000e-05,3.500000e-03
5,601988.SS,EGARCH,6.404000e-01,2.600000e-03,7.000000e-06,1.800000e-03
34,000776.SZ,GJR,6.350000e-01,5.000000e-03,2.500000e-05,3.600000e-03
9,601288.SS,GARCH,6.329000e-01,2.500000e-03,6.000000e-06,1.800000e-03
10,601288.SS,GJR,6.249000e-01,2.600000e-03,7.000000e-06,1.800000e-03
3,601988.SS,GARCH,6.200000e-01,2.700000e-03,7.000000e-06,1.900000e-03
30,601688.SS,GARCH,6.142000e-01,4.100000e-03,1.700000e-05,3.300000e-03
8,601939.SS,EGARCH,6.009000e-01,3.100000e-03,9.000000e-06,2.400000e-03
31,601688.SS,GJR,5.938000e-01,4.200000e-03,1.800000e-05,3.400000e-03
21,601628.SS,GARCH,5.654000e-01,4.900000e-03,2.400000e-05,3.400000e-03


In [10]:
def train_fusion_model(df, model_name, ts_feature="GARCH_pred", ticker="TICKER"):
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from catboost import CatBoostRegressor
    from lightgbm import LGBMRegressor
    from xgboost import XGBRegressor

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    if model_name not in models:
        raise ValueError(f"Model '{model_name}' not recognized.")

    df = df.copy()

    # Flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if ts_feature not in df.columns:
        raise ValueError(f"'{ts_feature}' not found in DataFrame for {ticker}")

    feature_cols = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D', 'Vol_t_1', 'Vol_t_2', 'Vol_t_3',
        'Vol_t_4', 'Vol_t_5', 'Vol_t_6', ts_feature
    ]

    X = df[feature_cols].copy()
    y = df["Vol_target"]

    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = evaluate_series(y_test, y_pred)

    print(f"✅ {model_name} + {ts_feature} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return metrics

In [11]:
import os
import pandas as pd

# Reuse existing forecast function
for ticker in tqdm(garch_data.keys()):
    for model_type in ["GJR", "EGARCH"]:
        print(f"\n📈 Adding {model_type}_pred to {ticker}...")
        df = garch_data[ticker]

        try:
            df, _ = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=False)
            garch_data[ticker] = df  # Update with new column
        except Exception as e:
            print(f"⚠️ {model_type} failed for {ticker}: {e}")

def train_all_fusion_models(garch_data_dict, results_path="china_fusion_model_results.csv"):
    # Load existing results if file exists
    if os.path.exists(results_path):
        print(f"📦 Loading saved fusion model results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for ts_feature in ["GARCH_pred", "GJR_pred", "EGARCH_pred"]:
            if ts_feature not in df.columns:
                print(f"⚠️ Skipping {ticker} - missing {ts_feature}")
                continue

            for model_name in ["RandomForest", "XGBoost", "CatBoost", "AdaBoost", "KNN"]:
                try:
                    metrics = train_fusion_model(df, model_name, ts_feature=ts_feature, ticker=ticker)
                    results.append({
                        "Stock": ticker,
                        "Fusion_Model": f"{ts_feature}+{model_name}",
                        "R2": round(metrics["R2"], 4),
                        "RMSE": round(metrics["RMSE"], 4),
                        "MSE": round(metrics["MSE"], 6),
                        "MAE": round(metrics["MAE"], 4),
                    })
                except Exception as e:
                    print(f"⚠️ {ticker} {ts_feature}+{model_name} failed: {e}")

    fusion_df = pd.DataFrame(results)
    fusion_df.to_csv(results_path, index=False)
    print(f"💾 Saved fusion model results to {results_path}")

    return fusion_df

fusion_results_df = train_all_fusion_models(garch_data)
fusion_results_sorted = fusion_results_df.sort_values(by="R2", ascending=False)
display(fusion_results_sorted)

  0%|          | 0/12 [00:00<?, ?it/s]


📈 Adding GJR_pred to 601398.SS...

📈 Adding EGARCH_pred to 601398.SS...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
  8%|▊         | 1/12 [00:35<06:26, 35.15s/it]


📈 Adding GJR_pred to 601988.SS...

📈 Adding EGARCH_pred to 601988.SS...


 17%|█▋        | 2/12 [01:10<05:53, 35.35s/it]

...

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p


📈 Adding GJR_pred to 000776.SZ...

📈 Adding EGARCH_pred to 000776.SZ...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/

✅ RandomForest + GARCH_pred for 601398.SS — R²: 0.8882, RMSE: 0.0014, MAE: 0.0009
✅ XGBoost + GARCH_pred for 601398.SS — R²: 0.8544, RMSE: 0.0016, MAE: 0.0010
✅ CatBoost + GARCH_pred for 601398.SS — R²: 0.8656, RMSE: 0.0016, MAE: 0.0011
✅ AdaBoost + GARCH_pred for 601398.SS — R²: 0.7935, RMSE: 0.0019, MAE: 0.0014
✅ KNN + GARCH_pred for 601398.SS — R²: 0.0676, RMSE: 0.0041, MAE: 0.0030
✅ RandomForest + GJR_pred for 601398.SS — R²: 0.8914, RMSE: 0.0014, MAE: 0.0009
✅ XGBoost + GJR_pred for 601398.SS — R²: 0.8580, RMSE: 0.0016, MAE: 0.0010
✅ CatBoost + GJR_pred for 601398.SS — R²: 0.8725, RMSE: 0.0015, MAE: 0.0010
✅ AdaBoost + GJR_pred for 601398.SS — R²: 0.7744, RMSE: 0.0020, MAE: 0.0013
✅ KNN + GJR_pred for 601398.SS — R²: 0.0676, RMSE: 0.0041, MAE: 0.0030
✅ RandomForest + EGARCH_pred for 601398.SS — R²: 0.8775, RMSE: 0.0015, MAE: 0.0010
✅ XGBoost + EGARCH_pred for 601398.SS — R²: 0.8554, RMSE: 0.0016, MAE: 0.0010
✅ CatBoost + EGARCH_pred for 601398.SS — R²: 0.8676, RMSE: 0.0016, MAE: 0

,Stock,Fusion_Model,R2,RMSE,MSE,MAE
38,601939.SS,GJR_pred+AdaBoost,0.9454,0.0009,0.000001,0.0007
43,601939.SS,EGARCH_pred+AdaBoost,0.9423,0.0009,0.000001,0.0008
33,601939.SS,GARCH_pred+AdaBoost,0.9420,0.0010,0.000001,0.0007
35,601939.SS,GJR_pred+RandomForest,0.9361,0.0010,0.000001,0.0008
42,601939.SS,EGARCH_pred+CatBoost,0.9310,0.0010,0.000001,0.0008
...,...,...,...,...,...,...
69,600036.SS,GJR_pred+KNN,-1.2963,0.0078,0.000061,0.0062
74,600036.SS,EGARCH_pred+KNN,-1.2963,0.0078,0.000061,0.0062
169,000776.SZ,GARCH_pred+KNN,-2.1375,0.0106,0.000113,0.0092
174,000776.SZ,GJR_pred+KNN,-2.1375,0.0106,0.000113,0.0092
